<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Roberta_Domain_Training_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This is a domain pre-training example.  It uses

# To Do

*   Check for the correct evaluation metric from the paper
*   Include Perplexity?

# Questions

*   For Training Complete Domain Setting, should we use the condensed model (putting as many reviews in the token window) to expedite training?  I think yes, just use 25M reviews and I think the training is roughly equivelant.
*   What is the correct domain training metric?




# Setup

In [ ]:
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━

I think it helps if you create an account on huggingface.  With that you can create an access token.  

This shows how to add the access token you created to HF_TOKEN in secrets in colab, per this website guide:
https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0


This shows how to share datasets (and Models) to hugging face
https://huggingface.co/docs/datasets/en/upload_dataset

I created a very small local Amazon Review dataset and uploaded it to HuggingFace. You should be able to load it from "BigTMiami/amazon_review_example_tokens_only_dataset".  You can see how I created it in the notebook Amazon Dataset Creation Example.ipynb in GitHub

In [ ]:
from datasets import load_dataset
dataset = load_dataset("BigTMiami/amazon_review_example_tokens_only_dataset")

Generating train split:   0%|          | 0/197 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/197 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizer.model_max_length
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512


# Train

In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM

In [ ]:
config = AutoConfig.from_pretrained("roberta-base")

In [ ]:
model = AutoModelForMaskedLM.from_config(config)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    f"small_training_test_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    # push_to_hub=True
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Before Training")
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Before Training
Perplexity: 58914.91


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,9.445504
2,No log,9.127286
3,No log,9.007509


TrainOutput(global_step=75, training_loss=9.4502392578125, metrics={'train_runtime': 33.7447, 'train_samples_per_second': 17.514, 'train_steps_per_second': 2.223, 'total_flos': 155589888807936.0, 'train_loss': 9.4502392578125, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"After Training")
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

After Training
Perplexity: 8106.03
